Connecting to Google Drive 

In [1]:
def connect_to_drive():
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

!pip install import_ipynb 
import import_ipynb 
connect_to_drive()
%cd '/content/gdrive/My Drive/Bioinformatics2'

Mounted at /content/gdrive
/content/gdrive/My Drive/Bioinformatics2


Importing modules

In [10]:
from utils import *
from VascularDataset import *
import torch
from utils import *
install_requirements()
from VascularDataset import *
import albumentations as A
from torchvision import transforms
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.augmentations.transforms import *
import segmentation_models_pytorch as smp
import wandb
from tqdm import tqdm
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR

In [4]:
import albumentations as A
from torchvision.transforms import *
from torch.utils.data import DataLoader
import torch
import albumentations
from albumentations.augmentations.functional import elastic_transform_approx
import random

input_path_training='/content/gdrive/.shortcut-targets-by-id/1mtNVywqwmtCBefeGQrY2c6F6xBQQWq30/vascular_segmentation/Train'
input_path_test='/content/gdrive/.shortcut-targets-by-id/1mtNVywqwmtCBefeGQrY2c6F6xBQQWq30/vascular_segmentation/Test'

txt_with_training_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/training_paths_inputs.txt'
txt_with_training_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/training_paths_targets.txt'
txt_with_validation_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_inputs.txt'
txt_with_validation_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_targets.txt'
txt_with_test_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/test_paths_inputs.txt'
txt_with_test_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/test_paths_targets.txt'

imgs_training_input_split=read_list_images_path_from_txt(txt_with_training_paths_inputs)
imgs_training_target_split=read_list_images_path_from_txt(txt_with_training_paths_targets)
imgs_validation_input_split=read_list_images_path_from_txt(txt_with_validation_paths_inputs)
imgs_validation_target_split=read_list_images_path_from_txt(txt_with_validation_paths_targets)
imgs_test_input=read_list_images_path_from_txt(txt_with_test_paths_inputs)
imgs_test_target=read_list_images_path_from_txt(txt_with_test_paths_targets)

training_dataset_plain=VascularDataset(imgs_training_input_split, imgs_training_target_split)
validation_dataset=VascularDataset(imgs_validation_input_split, imgs_validation_target_split)
test_dataset=VascularDataset(imgs_test_input, imgs_test_target)

save_model_folder='/content/gdrive/MyDrive/Bioinformatics2/saved_models/'

batch_size=32

print(f'Training dataset length: {len(training_dataset_plain)}')
print(f'Validation dataset length: {len(validation_dataset)}')
print(f'Test dataset length: {len(test_dataset)}')

valid_dataloader=DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1)

Training dataset length: 160
Validation dataset length: 40
Test dataset length: 10


Defining dataset and dataloaders

In [5]:
affine_transform=A.Compose([Flip(), ShiftScaleRotate()])
elastic_transform=A.Compose([ElasticTransform()])
pixel_wise_transform=A.Compose([GaussNoise(), HueSaturationValue(), RandomBrightnessContrast(), RandomGamma()])

Defining hyper-parameters

In [11]:
project_name='Bioinformatics_segmentation_sweep_1'

sweep_config = {
  "name" : "qhzcjnzi",
  "method" : "bayes",
  "parameters" : 
  {
      "optimizer" : {
      "values": ['SGD', 'Adam']},
   
    "learning_rate" :{
      "min": 0.01,
      "max": 0.2},

  "encoder_type":{
      "values": ['resnet18', 'resnet34', 'resnet50', 'resnext50_32x4d']},
   
  "step_size":{
      "min": 1,
      "max": 20},
   
  "gamma": {
      "min":0.1,
      "max":0.95},
   
   "n_epochs":{
       "min":15,
       "max":65},
   
   "transforms":{
       "values": [None, affine_transform, elastic_transform, pixel_wise_transform]
  }
  },
  
    "metric":{
      "name": "validation loss",
      "goal": "minimize"},

    "early_terminate":{
      "type": "hyperband",
      "min_iter": 2}
}

sweep_id = wandb.sweep(sweep_config)
save_model_path='/content/gdrive/MyDrive/Bioinformatics2/saved_models/'+'Unet_'+sweep_id
print(f'Path for model saving: {save_model_path}')

Create sweep with ID: 3agguswv
Sweep URL: https://wandb.ai/emafasce/uncategorized/sweeps/3agguswv
Path for model saving: /content/gdrive/MyDrive/Bioinformatics2/saved_models/Unet_3agguswv


Defining wandb training

In [8]:
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

loss=torch.nn.BCEWithLogitsLoss()

def iou_numpy(outputs: np.array, labels: np.array):
    intersection = torch.logical_and(labels, outputs)
    union = torch.logical_or(labels, outputs)
    iou_score = torch.sum(intersection) / torch.sum(union)
    return iou_score 

def train(config=None):

    with wandb.init(config=config) as run:
        config=wandb.config
        
        optimizer_config=config["optimizer"]
        encoder_type=config['encoder_type']
        learning_rate=config["learning_rate"]
        step_size=config["step_size"]
        gamma=config["gamma"]
        n_epochs=config["n_epochs"]
        transforms_config=config["transforms"]

        training_dataset=VascularDataset(imgs_training_input_split, imgs_training_target_split)
        train_dataloader=DataLoader(training_dataset, batch_size=batch_size,num_workers=2, shuffle=True, pin_memory=True)

        #Loading the model
        model = smp.Unet(
          encoder_name=encoder_type, encoder_weights="imagenet",in_channels=3,classes=1)
        model.cuda()
        
        #Loading optimizers
        if optimizer_config=='SGD':
          optimizer=SGD(model.parameters(), lr=learning_rate)
        if optimizer_config=='Adam':
          optimizer=Adam(model.parameters(), lr=learning_rate)

        scheduler=StepLR(optimizer, step_size=step_size, gamma=gamma)
        lowest_validation_loss=9999

        for epoch in range(1, n_epochs+1,1):

          #Training phase
          model.train()
          training_loss=0
          training_iou=0
          training_samples=0

          for x,y in tqdm(train_dataloader): 
            optimizer.zero_grad()
            out=model(x.cuda())
            myloss=loss(out, y.cuda())

            for out_i,y_i in zip(out,y.cuda()):
              iou=iou_numpy(out_i>0,y_i)
              training_iou+=iou
              training_samples+=1

            myloss.backward()
            optimizer.step()
            training_loss+=myloss.item()
          
          print(f'Epoch #{epoch}, training loss: {training_loss/training_samples}, training iou: {training_iou/training_samples}')
          scheduler.step()

          #Validation phase
          model.eval()
          validation_loss=0
          validation_iou=0
          validation_samples=0

          for x,y in tqdm(valid_dataloader): 
            out=model(x.cuda())
            myloss=loss(out, y.cuda())

            for out_i,y_i in zip(out,y.cuda()):
              iou=iou_numpy(out_i>0,y_i)
              validation_iou+=iou
              validation_samples+=1

            validation_loss+=myloss.item()

          print(f'Epoch #{epoch}, validation loss: {validation_loss/validation_samples}, validation iou: {validation_iou/validation_samples}')

          if validation_loss/validation_samples<lowest_validation_loss:
            lowest_validation_loss=validation_loss/validation_samples
            torch.save(model,f'{save_model_path}.pth')
            last_epoch_saved=epoch
          
          wandb.log({"training loss": training_loss/training_samples, "validation loss": validation_loss/validation_samples, "epoch":epoch})
          wandb.log({"training iou": training_iou/training_samples, "validation iou": validation_iou/validation_samples, "epoch": epoch})

        print(f'Last epoch saved: {last_epoch_saved}')
        wandb.log({"last epoch saved": last_epoch_saved})


Launching wandb

In [ ]:
#sweep_id="qhzcjnzi"
project_name="Bioinformatics_segmentation_sweep_2"
wandb.init(project=project_name)
count=10
wandb.agent(sweep_id=sweep_id, function=train, count=count)

wandb: WARNING Ignored wandb.init() arg project when running a sweep


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 9iecxgpk with config:
wandb: 	encoder_type: resnet34
wandb: 	gamma: 0.7440585142093782
wandb: 	learning_rate: 0.17414321807389815
wandb: 	n_epochs: 16
wandb: 	optimizer: Adam
wandb: 	step_size: 16
wandb: 	transforms: {'bbox_format': None, 'bbox_label_fields': [], 'bboxes_name': 'bboxes', 'keypoints_format': None, 'keypoints_label_fields': [], 'keypoints_name': 'keypoints', 'p': 1, 'params': {'bboxes': {}, 'keypoints': {}}, 'transforms': [{'_additional_targets': {}, 'always_apply': False, 'p': 0.5, 'var_limit': [10, 50]}, {'_additional_targets': {}, 'always_apply': False, 'hue_shift_limit': [-20, 20], 'p': 0.5, 'sat_shift_limit': [-30, 30], 'val_shift_limit': [-20, 20]}, {'_additional_targets': {}, 'always_apply': False, 'brightness_limit': [-0.2, 0.2], 'contrast_limit': [-0.2, 0.2], 'p': 0.5}, {'_additional_targets': {}, 'always_apply': False, 'gamma_limit': [80, 120], 'p': 0.5}]}


100%|██████████| 5/5 [00:53<00:00, 10.63s/it]


Epoch #1, training loss: 0.030425212481070097, training iou: 0.06912689656019211


100%|██████████| 2/2 [00:14<00:00,  7.11s/it]


...

100%|██████████| 2/2 [00:15<00:00,  7.65s/it]


Epoch #15, validation loss: 0.029602967364127035, validation iou: 0.0


100%|██████████| 5/5 [00:54<00:00, 10.88s/it]


Epoch #16, training loss: 0.019340363867920308, training iou: 0.0


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#https://github.com/wandb/client/issues/1501
qhzcjnzi
#Cambiare savepath (salva la sweepID e non la runID)
#Cambiare numero di workers nel dataloader